In [446]:
import panel as pn
pn.extension()
import pandas as pd
import numpy as np
import hvplot.pandas
import param, random
import datetime as dt


# Helpers
HOURS = 60 * 60
DAYS = 24 * HOURS
WEEKS = 7 * DAYS
ONE_HUNDRED_PERCENT = 1e18
ONE_TOKEN = 1e18
FUNDRAISING_ONE_HUNDRED_PERCENT = 1e6
FUNDRAISING_ONE_TOKEN = 1e18
assert(1e-8 * FUNDRAISING_ONE_TOKEN == 10000000000.0)
HOLDERS_PER_TRANSACTION = 10
PPM = 1000000
BLOCKTIME = 5
# const CONTRIBUTORS_PROCESSED_PER_TRANSACTION = 10

In [447]:
# This is the amount of wxDAI that can be staked to create TEC hatch tokens - TEC hatch upgrades into a commons
HATCH_ORACLE_RATIO = 0.005 

In [448]:
# Dandelion Voting Settings

# Used for administrative or binary choice decisions with ragequit-like functionality
SUPPORT_REQUIRED = 0.6 * ONE_HUNDRED_PERCENT
MIN_ACCEPTANCE_QUORUM = 0.02 * ONE_HUNDRED_PERCENT
VOTE_DURATION_BLOCKS = 3 * DAYS / BLOCKTIME
VOTE_BUFFER_BLOCKS = 8 * HOURS / BLOCKTIME
VOTE_EXECUTION_DELAY_BLOCKS = 24 * HOURS / BLOCKTIME
VOTING_SETTINGS = [SUPPORT_REQUIRED, MIN_ACCEPTANCE_QUORUM, VOTE_DURATION_BLOCKS, VOTE_BUFFER_BLOCKS, VOTE_EXECUTION_DELAY_BLOCKS]
# Set the fee paid to the org to create an administrative vote
TOLLGATE_FEE = 3 * ONE_TOKEN

In [449]:
class DandelionVoting(param.Parameterized):
    support_required = param.Number(0.6, bounds=(0.5,1), step=0.01)
    min_acceptance_quorum = param.Number(0.02, bounds=(0,1), step=0.01) # Must be less or equal to support required
    time_it_takes_for_a_proposal_to_pass = param.Integer(3 + 8 + 24, bounds=(0,None))
#     vote_buffer_blocks = param.Integer(8, bounds=(0,None))
#     vote_execution_delay_blocks = param.Integer(24, bounds=(0,None))
    cost_to_make_a_proposal = param.Number(3, doc="cost to make a proposal")
    

In [450]:
dandelion = DandelionVoting()

In [451]:
pn.Pane(dandelion)

Param(DandelionVoting)

In [452]:
class Hatch(param.Parameterized):
    # CSTK Ratio
    total_cstk_tokens = param.Number(700000, constant=True)
    hatch_oracle_ratio = param.Number(0.005, constant=True)
    @param.depends('hatch_oracle_ratio', 'total_cstk_tokens')
    def wxdai_range(self):
        return pn.Row(pn.Pane("Cap on wxdai staked: "), self.hatch_oracle_ratio * self.total_cstk_tokens)
    
    # Min and Target Goals
    min_goal = param.Number(5, bounds=(1,100), step=10)
    max_goal = param.Number(1000, bounds=(100,10000), step=50) # Something to consider -> target goal or max goal
    
    # Hatch params
    hatch_period = param.Integer(15, bounds=(5, 30), step=2)
    hatch_exchange_rate = param.Number() # This needs to be tested and explained -> See the forum post
    hatch_tribute = param.Number(0.05, bounds=(0,1))    


In [453]:
pn.Pane(Hatch)

Param(ParameterizedMetaclass, name='Hatch')

### Impact Hours

In [454]:
import pandas as pd
import panel as pn
import os
import hvplot.pandas
APP_PATH = './'

sheets = [
    "Total Impact Hours so far", 
    "IH Predictions", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#5 Nov 20",
    "#4 Nov 6",
    "#3 Oct 23",
    "#2 Oct 9",
    "#1 Sept 24",
    "#0 Sept 7 (historic)",
] + [f"#{i} IH Results" for i in range(9)]
sheets = {i:sheet for i, sheet in enumerate(sheets)}

def read_excel(sheet_name="Total Impact Hours so far", header=1, index_col=0, usecols=None) -> pd.DataFrame:
    data = pd.read_excel(
        os.path.join(APP_PATH, "data", "TEC Praise Quantification.xlsx"),
        sheet_name=sheet_name,
        engine='openpyxl',
        header=header,
        index_col=index_col,
        usecols=usecols
    ).reset_index().dropna(how='any')
    return data

## Tests
total_impact_hours = read_excel()
impact_hour_data = read_excel(sheet_name="IH Predictions", header=0, index_col=0, usecols='A:I').drop(index=19)
pn.Row(impact_hour_data.hvplot.table(), total_impact_hours.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [455]:
import numpy as np
import holoviews as hv

In [456]:
class Beta(param.Parameterized):
    a = param.Number(1, (1,10))
    b = param.Number(10, bounds=(1,100), step=0.5)
    size = param.Integer(1000, bounds=(100,10000))
    
    @param.depends('a','b','size')
    def view(self):
        data = pd.DataFrame(np.random.beta(self.a, self.b, self.size))
        return data.hvplot.kde()

In [457]:
b = Beta()

In [458]:
pn.Row(b, b.view)

Row
    [0] Column(margin=5, name='Beta', width=300)
        [0] StaticText(value='<b>Beta</b>')
        [1] FloatSlider(end=10, name='A', start=1, value=1, value_throttled=1)
        [2] FloatSlider(end=100, name='B', start=1, step=0.5, value=10, value_throttled=10)
        [3] IntSlider(end=10000, name='Size', start=100, value=1000, value_throttled=1000)
    [1] ParamMethod(method)

In [459]:
class HoursDistribution(param.Parameterized):
    impact_hours = param.DataFrame(read_excel())
    mean_hours_prediction = param.Number()
#     mean_hours = impact_hours['Impact Hours'].mean()
#     std_hours = impact_hours['Impact Hours'].std()
#     mean_hours_prediction = param.Number(mean_hours, bounds=(mean_hours - 2*std_hours, mean_hours + 2*std_hours))

    def __init__(self, **params):
        super(HoursDistribution, self).__init__(**params)
        mean_hours = self.impact_hours['Impact Hours'].mean()
        std_hours = self.impact_hours['Impact Hours'].std()
        self.mean_hours_prediction = mean_hours
        self.param['mean_hours_prediction'].bounds = (mean_hours - 2*std_hours, mean_hours + 2*std_hours)   
        
    @param.depends('mean_hours_prediction')
    def view(self):
        return self.impact_hours['Impact Hours'].hvplot.kde(xlim=(0,None))

In [460]:
a = HoursDistribution()

In [461]:
a.impact_hours

,Telegram Handle,Impact Hours
0,sembrestels,233.570354
1,santigs67,207.412308
2,zeptimusq,160.483885
3,cranders71,135.760369
4,akrtws,131.120931
...,...,...
194,wolfofcurrency,0.037712
195,llll79,0.027621
196,burningfiat,0.025141
197,immortaz,0.025141


In [462]:
pn.Row(a.view)

Row
    [0] ParamMethod(method)

In [463]:
total_impact_hours.sum()

Telegram Handle    sembrestelssantigs67zeptimusqcranders71akrtwsy...
Impact Hours                                             2893.266082
dtype: object

In [464]:
class FillBar(param.Parameterized):
    
    amount = param.Number()
    
    def __init__(self, label: str, amount: float, bounds: tuple, **params):
        super(FillBar, self).__init__(**params)
        self.label = label
        self.amount = amount
        self.param['amount'].bounds = bounds
        
    
    @param.depends('amount')
    def view(self):
        bar = pd.DataFrame([(self.label, self.amount)], columns=['Label', 'Amount'])
        return bar.hvplot.bar(ylim=(0, self.param['amount'].bounds[1]), x='Label', y='Amount')
        

In [465]:
f = FillBar('test', 5, (0, 10))

In [466]:
pn.Row(f, f.view)

Row
    [0] Column(margin=5, name='FillBar', width=300)
        [0] StaticText(value='<b>FillBar</b>')
        [1] FloatSlider(end=10, name='Amount', value=5, value_throttled=5)
    [1] ParamMethod(method)

In [467]:
B = pn.widgets.FloatSlider(name='Beta Distribution Parameter', start=0.01, end=1, step=0.02, value=0.01)
N = pn.widgets.FloatSlider(name='Number of Members', start=5, end=100, step=1, value=5)


In [468]:
@pn.depends(B, N)
def beta_dist(B, N):
    xs = range(1,N)
    y = [B*(1-B)**t for t in xs]
    # Convert to a dataframe and return
    distribution = pd.DataFrame([xs,y]).T
    distribution.columns = ['Ranking','Impact Hours']
    # Force dist to sum to one
    distribution['Impact Hours'] += (1 - distribution['Impact Hours'].sum())/len(distribution)
    distribution.iloc[-1] += (1 - distribution['Impact Hours'].sum())
    
    return pn.Row(
                distribution.set_index("Ranking"),
                distribution.hvplot.line(x='Ranking',y='Impact Hours', title="Cultural Tribute Distribution"),
#                 capital.hvplot.line(xlabel='Weeks', title="Working Capital")
            )

In [469]:
pn.Column(B, N, beta_dist)

Column
    [0] FloatSlider(name='Beta Distribution P..., start=0.01, step=0.02, value=0.01, value_throttled=0.01)
    [1] FloatSlider(end=100, name='Number of Members', start=5, step=1, value=5, value_throttled=5)
    [2] ParamFunction(function)

In [470]:
data = pd.read_csv('data/IHPredictions.csv').query('Model=="Historic"')
optimistic_data = pd.read_csv('data/IHPredictions.csv').query('Model=="Optimistic"')
pn.Row(data.hvplot.table(), optimistic_data.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [471]:
class ImpactHours(param.Parameterized):
    predicted = pd.read_csv('data/IHPredictions.csv').query('Model=="Historic"')
    optimistic =  pd.read_csv('data/IHPredictions.csv').query('Model=="Optimistic"')
    prediction = param.Number(0, bounds=(0,1), step=0.01)
    
    @param.depends('prediction')
    def view(self):
        x = 'End Date'
        y_curve = 'Total IH'
        y_bar = 'Total Hours'
        
        actual_curve = self.predicted.hvplot(x, y_curve, rot=45, title='Impact Hours Accumulation Curve :)')
        actual_bar = self.predicted.hvplot.bar(x, y_bar, label='Actual')
        
        predicted = self.predicted[self.predicted["Actual / Predicted"] == "Predicted"].copy()
        predicted[y_curve] = 0.5 * predicted[y_curve] + 0.5 * self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"][y_curve]
        
        predicted_curve = predicted.hvplot(x, y_curve, rot=45, title='Impact Hours Accumulation Curve :)')
        predicted_bar = predicted.hvplot.bar(x, y_bar, label='Predicted')
        
        optimistic_curve = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].hvplot(x, y_curve)
        optimistic_bar = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].hvplot.bar(x, y_bar, label='Optimistic')
        
        return pn.Row(actual_curve * actual_bar * predicted_curve * predicted_bar * optimistic_curve * optimistic_bar, predicted.hvplot.table()) 

In [472]:
i = ImpactHours()

In [473]:
self = i

In [474]:
x = 'End Date'
y_curve = 'Total IH'
y_bar = 'Total Hours'

actual_curve = self.predicted.hvplot(x, y_curve, rot=45, title='Impact Hours Accumulation Curve :)')
actual_bar = self.predicted.hvplot.bar(x, y_bar, label='Actual')

predicted = self.predicted[self.predicted["Actual / Predicted"] == "Predicted"].copy()
predicted[y_curve] = 0.5 * predicted[y_curve].reset_index() + 0.5 * self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"][y_curve]

In [475]:
predicted = self.predicted[self.predicted["Actual / Predicted"] == "Predicted"].copy()

In [476]:
predicted

,Round,Total Hours,Actual / Predicted,End Date,Total IH,Min Goal @ 15 DAI/IH & 15% Tribute,Min Goal @ 10 IH/hr & 15% Tribute,Min Goal @ 10 IH/hr & 20% Tribute,Change in Impact hours from previous round,Model
9,9,574.526090,Predicted,"Fri, Jan 15",3437.911390,"$343,791.14","$229,194.09","$171,895.57",305.407329,Historic
10,10,649.052180,Predicted,"Fri, Jan 29",4086.963570,"$408,696.36","$272,464.24","$204,348.18",74.526090,Historic
11,11,723.578270,Predicted,"Fri, Feb 12",4810.541839,"$481,054.18","$320,702.79","$240,527.09",74.526090,Historic
12,12,798.104360,Predicted,"Fri, Feb 26",5608.646199,"$560,864.62","$373,909.75","$280,432.31",74.526090,Historic
13,13,872.630450,Predicted,"Fri, Mar 12",6481.276649,"$648,127.66","$432,085.11","$324,063.83",74.526090,Historic
14,14,947.156540,Predicted,"Fri, Mar 26",7428.433188,"$742,843.32","$495,228.88","$371,421.66",74.526090,Historic
15,15,1021.682629,Predicted,"Fri, Apr 9",8450.115817,"$845,011.58","$563,341.05","$422,505.79",74.526090,Historic


In [477]:
0.5 * predicted[y_curve] + 0.5 * self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"][y_curve]

9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
Name: Total IH, dtype: float64

In [478]:
0.5 * self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"][y_curve]

25    1681.69265
26    1931.69265
27    2181.69265
28    2431.69265
29    2681.69265
30    2931.69265
31    3181.69265
Name: Total IH, dtype: float64

In [479]:
predicted

,Round,Total Hours,Actual / Predicted,End Date,Total IH,Min Goal @ 15 DAI/IH & 15% Tribute,Min Goal @ 10 IH/hr & 15% Tribute,Min Goal @ 10 IH/hr & 20% Tribute,Change in Impact hours from previous round,Model
9,9,574.526090,Predicted,"Fri, Jan 15",3437.911390,"$343,791.14","$229,194.09","$171,895.57",305.407329,Historic
10,10,649.052180,Predicted,"Fri, Jan 29",4086.963570,"$408,696.36","$272,464.24","$204,348.18",74.526090,Historic
11,11,723.578270,Predicted,"Fri, Feb 12",4810.541839,"$481,054.18","$320,702.79","$240,527.09",74.526090,Historic
12,12,798.104360,Predicted,"Fri, Feb 26",5608.646199,"$560,864.62","$373,909.75","$280,432.31",74.526090,Historic
13,13,872.630450,Predicted,"Fri, Mar 12",6481.276649,"$648,127.66","$432,085.11","$324,063.83",74.526090,Historic
14,14,947.156540,Predicted,"Fri, Mar 26",7428.433188,"$742,843.32","$495,228.88","$371,421.66",74.526090,Historic
15,15,1021.682629,Predicted,"Fri, Apr 9",8450.115817,"$845,011.58","$563,341.05","$422,505.79",74.526090,Historic


In [480]:
i = ImpactHours()
pn.Row(i, i.view)

Row
    [0] Column(margin=5, name='ImpactHours', width=300)
        [0] StaticText(value='<b>ImpactHours</b>')
        [1] FloatSlider(name='Prediction', step=0.01, value_throttled=0)
    [1] ParamMethod(method)

In [481]:
ih_cumulative_curve = data.hvplot(x='End Date', y='Total IH', rot=45, title='Impact Hours Accumulation Curve :)')


In [482]:

ih_cumulative_curve = data.hvplot(x='End Date', y='Total IH', rot=45, title='Impact Hours Accumulation Curve :)')

ih_bar = data.hvplot.bar(x='End Date', y='Total Hours', label='Actual')

predicted_ih_cumulative_curve = data[data["Actual / Predicted"] == "Predicted"].hvplot(x='End Date', y='Total IH', rot=45, title='Impact Hours Accumulation Curve :)')

predicted_ih_bar = data[data["Actual / Predicted"] == "Predicted"].hvplot.bar(x='End Date', y='Total Hours', label='Predicted')

optimistic_ih_curve = optimistic_data[optimistic_data["Actual / Predicted"] == "Predicted"].hvplot(x='End Date', y='Total IH')

optimistic_ih_bars = optimistic_data[optimistic_data["Actual / Predicted"] == "Predicted"].hvplot.bar(x='End Date', y='Total Hours', label='Optimistic')

ih_cumulative_curve * ih_bar * predicted_ih_cumulative_curve * predicted_ih_bar * optimistic_ih_curve * optimistic_ih_bars

:Overlay
   .Curve.I         :Curve   [End Date]   (Total IH)
   .Bars.Actual     :Bars   [End Date]   (Total Hours)
   .Curve.II        :Curve   [End Date]   (Total IH)
   .Bars.Predicted  :Bars   [End Date]   (Total Hours)
   .Curve.III       :Curve   [End Date]   (Total IH)
   .Bars.Optimistic :Bars   [End Date]   (Total Hours)

Slider Distribution of Impact Hours (Beta Distribution?)

Slider of number of hatchers

Bar Charts of Payment Rankings

Pie Chart of Payments

In [483]:
optimistic_data

,Round,Total Hours,Actual / Predicted,End Date,Total IH,Min Goal @ 15 DAI/IH & 15% Tribute,Min Goal @ 10 IH/hr & 15% Tribute,Min Goal @ 10 IH/hr & 20% Tribute,Change in Impact hours from previous round,Model
16,0 (Historic),408.060000,Actual,"Fri, Sep 11",408.060000,"$40,806.00","$27,204.00","$20,403.00",NaN,Optimistic
17,1,57.408000,Actual,"Fri, Sep 25",465.468000,"$46,546.80","$31,031.20","$23,273.40",NaN,Optimistic
18,2,159.390000,Actual,"Fri, Oct 9",624.858000,"$62,485.80","$41,657.20","$31,242.90",101.982000,Optimistic
19,3,273.315000,Actual,"Fri, Oct 23",898.173000,"$89,817.30","$59,878.20","$44,908.65",113.925000,Optimistic
20,4,273.290000,Actual,"Fri, Nov 6",1171.463000,"$117,146.30","$78,097.53","$58,573.15",-0.025000,Optimistic
21,5,404.560000,Actual,"Fri, Nov 20",1576.023000,"$157,602.30","$105,068.20","$78,801.15",131.270000,Optimistic
22,6,513.679000,Actual,"Fri, Dec 4",2089.702000,"$208,970.20","$139,313.47","$104,485.10",109.119000,Optimistic
23,7,504.564540,Actual,"Fri, Dec 18",2594.266539,"$259,426.65","$172,951.10","$129,713.33",-9.114461,Optimistic
24,8,269.118761,Actual,"Fri, Jan 1",2863.385300,"$286,338.53","$190,892.35","$143,169.26",-235.445779,Optimistic
25,9,500.000000,Predicted,"Fri, Jan 15",3363.385300,"$336,338.53","$224,225.69","$168,169.26",230.881239,Optimistic


In [484]:
data

,Round,Total Hours,Actual / Predicted,End Date,Total IH,Min Goal @ 15 DAI/IH & 15% Tribute,Min Goal @ 10 IH/hr & 15% Tribute,Min Goal @ 10 IH/hr & 20% Tribute,Change in Impact hours from previous round,Model
0,0 (Historic),408.060000,Actual,"Fri, Sep 11",408.060000,"$40,806.00","$27,204.00","$20,403.00",NaN,Historic
1,1,57.408000,Actual,"Fri, Sep 25",465.468000,"$46,546.80","$31,031.20","$23,273.40",NaN,Historic
2,2,159.390000,Actual,"Fri, Oct 9",624.858000,"$62,485.80","$41,657.20","$31,242.90",101.982000,Historic
3,3,273.315000,Actual,"Fri, Oct 23",898.173000,"$89,817.30","$59,878.20","$44,908.65",113.925000,Historic
4,4,273.290000,Actual,"Fri, Nov 6",1171.463000,"$117,146.30","$78,097.53","$58,573.15",-0.025000,Historic
5,5,404.560000,Actual,"Fri, Nov 20",1576.023000,"$157,602.30","$105,068.20","$78,801.15",131.270000,Historic
6,6,513.679000,Actual,"Fri, Dec 4",2089.702000,"$208,970.20","$139,313.47","$104,485.10",109.119000,Historic
7,7,504.564540,Actual,"Fri, Dec 18",2594.266539,"$259,426.65","$172,951.10","$129,713.33",-9.114461,Historic
8,8,269.118761,Actual,"Fri, Jan 1",2863.385300,"$286,338.53","$190,892.35","$143,169.26",-235.445779,Historic
9,9,574.526090,Predicted,"Fri, Jan 15",3437.911390,"$343,791.14","$229,194.09","$171,895.57",305.407329,Historic


In [485]:
tota

NameError: name 'tota' is not defined

In [486]:
import numpy as np

h = Hatch()

class ImpactHours(param.Parameterized):
    max_ih_rate = param.Number(0.01, bounds=(0,200))
    expected_raise_per_ih = param.Number(0.012, bounds=(0,20))

    @param.depends('max_ih_rate', 'expected_raise_per_ih')
    def impact_hours_rewards(self):
        x = np.linspace(h.min_goal, h.max_goal)

        R = self.max_ih_rate

        m = self.expected_raise_per_ih
        
        H = total_impact_hours['Impact Hours'].sum()

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['x','y']

        return df.hvplot(x='x')

In [487]:
i = ImpactHours()

In [488]:
pn.Row(i,i.impact_hours_rewards)

Row
    [0] Column(margin=5, name='ImpactHours', width=300)
        [0] StaticText(value='<b>ImpactHours</b>')
        [1] FloatSlider(end=200, name='Max ih rate', value=0.01, value_throttled=0.01)
        [2] FloatSlider(end=20, name='Expected raise per ih', value=0.012, value_throttled=0.012)
    [1] ParamMethod(method)

In [489]:
h = Hatch()

In [490]:
h.min_goal

5

In [491]:
h.max_goal

1000

In [492]:
i = ImpactHours()

In [493]:
i.max_ih_rate

0.01

In [494]:
i.expected_raise_per_ih

0.012

In [495]:
// Max theoretical rate per impact hour in Collateral_token per IH
const MAX_IH_RATE = .01 * ONE_TOKEN
// How much will we need to raise to reach 1/2 of the MAX_IH_RATE divided by total IH
const EXPECTED_RAISE_PER_IH = .012 * ONE_TOKEN / 10 ** IMPACT_HOUR_DECIMALS

SyntaxError: invalid syntax (<ipython-input-495-3371eec0d7c4>, line 1)

In [496]:
total_impact_hours['Impact Hours'].sum()


2893.26608161608

In [497]:
impact_hour_data

,Round,Total Hours,Actual / Predicted,End Date,Total IH,Min Goal @ 15 DAI/IH & 15% Tribute,Min Goal @ 10 IH/hr & 15% Tribute,Min Goal @ 10 IH/hr & 20% Tribute,Change in Impact hours from previous round
2,2,159.390000,Actual,2020-10-09 00:00:00,624.858000,62485.8000,41657.20000,31242.9000,101.982000
3,3,273.315000,Actual,2020-10-23 00:00:00,898.173000,89817.3000,59878.20000,44908.6500,113.925000
4,4,273.290000,Actual,2020-11-06 00:00:00,1171.463000,117146.3000,78097.53333,58573.1500,-0.025000
5,5,404.560000,Actual,2020-11-20 00:00:00,1576.023000,157602.3000,105068.20000,78801.1500,131.270000
6,6,513.679000,Actual,2020-12-04 00:00:00,2089.702000,208970.2000,139313.46670,104485.1000,109.119000
7,7,504.564539,Actual,2020-12-18 00:00:00,2594.266539,259426.6539,172951.10260,129713.3270,-9.114461
8,8,269.118760,Actual,2021-01-01 00:00:00,2863.385300,286338.5300,190892.35330,143169.2650,-235.445779
9,9,574.526090,Predicted,2021-01-15 00:00:00,3437.911390,343791.1390,229194.09270,171895.5695,305.407329
10,10,649.052180,Predicted,2021-01-29 00:00:00,4086.963570,408696.3570,272464.23800,204348.1785,74.526090
11,11,723.578270,Predicted,2021-02-12 00:00:00,4810.541839,481054.1839,320702.78930,240527.0920,74.526090


In [498]:
total_impact_hours

,Telegram Handle,Impact Hours
0,sembrestels,233.570354
1,santigs67,207.412308
2,zeptimusq,160.483885
3,cranders71,135.760369
4,akrtws,131.120931
...,...,...
194,wolfofcurrency,0.037712
195,llll79,0.027621
196,burningfiat,0.025141
197,immortaz,0.025141


In [499]:
hatch = Hatch()

In [500]:
pn.Row(hatch, hatch.wxdai_range)

Row
    [0] Column(margin=5, name='Hatch', width=300)
        [0] StaticText(value='<b>Hatch</b>')
        [1] FloatInput(disabled=True, name='Total cstk tokens', value=700000, value_throttled=700000)
        [2] FloatInput(disabled=True, name='Hatch oracle ratio', value=0.005, value_throttled=0.005)
        [3] FloatSlider(end=100, name='Min goal', start=1, step=10, value=5, value_throttled=5)
        [4] FloatSlider(end=10000, name='Max goal', start=100, step=50, value=1000, value_throttled=1000)
        [5] IntSlider(end=30, name='Hatch period', start=5, step=2, value=15, value_throttled=15)
        [6] FloatInput(name='Hatch exchange rate', value_throttled=0.0)
        [7] FloatSlider(name='Hatch tribute', value=0.05, value_throttled=0.05)
    [1] ParamMethod(method)